In [ ]:
!pip install simpletransformers

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
df = pd.read_csv("/content/drive/My Drive/balanced_for_sentiment.csv")

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
df

,Sentiment,Unnamed: 1,Text,Sentiment.1
0,0,16345,I really wanted to like this soup. It's organi...,-1
1,0,31701,These are very very small and taste really bad...,-1
2,2,1375,I absolutely love Jelly Bellys. They are the o...,1
3,2,16905,This candy is just as good now as it was 20 ye...,1
4,1,11493,"The word ""sauce"" probably makes you think abou...",0
...,...,...,...,...
127915,2,23647,I love this snack. We still have a lot left. B...,1
127916,2,40557,I recently bought a Melitta One:One expecting ...,1
127917,1,7984,"When I drink this, it tastes like flavored hot...",0
127918,2,40827,Make sure you have a small dog...Big Dogs migh...,1


In [6]:
df.drop(columns=['Sentiment','Unnamed: 1'], inplace=True)
df.rename(columns = {'Sentiment.1':'Sentiment'}, inplace = True) 
df['Sentiment']=df['Sentiment']+1
df

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,Text,Sentiment
0,I really wanted to like this soup. It's organi...,0
1,These are very very small and taste really bad...,0
2,I absolutely love Jelly Bellys. They are the o...,2
3,This candy is just as good now as it was 20 ye...,2
4,"The word ""sauce"" probably makes you think abou...",1
...,...,...
127915,I love this snack. We still have a lot left. B...,2
127916,I recently bought a Melitta One:One expecting ...,2
127917,"When I drink this, it tastes like flavored hot...",1
127918,Make sure you have a small dog...Big Dogs migh...,2


In [7]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [8]:
model_args = ClassificationArgs(num_train_epochs=1)

In [9]:
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=3,
    args=model_args
) 

INFO:filelock:Lock 140229248970480 acquired on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock


INFO:filelock:Lock 140229248970480 released on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock


INFO:filelock:Lock 140229269345336 acquired on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


INFO:filelock:Lock 140229269345336 released on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

INFO:filelock:Lock 140229232967808 released on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


In [10]:
model.train_model(df_train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:338: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [11]:
from sklearn.metrics import f1_score, accuracy_score

In [12]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(df_test, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:810: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7231751112176641, 'f1': 0.8153924327704816, 'acc': 0.8153924327704816, 'eval_loss': 0.4759777503244211}


In [13]:
result

{'acc': 0.8153924327704816,
 'eval_loss': 0.4759777503244211,
 'f1': 0.8153924327704816,
 'mcc': 0.7231751112176641}

In [17]:
predictions, raw_outputs = model.predict(["Product is very good and awesome"])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [18]:
predictions

array([2])

In [19]:
raw_outputs

array([[-2.107 , -0.6963,  4.566 ]], dtype=float16)